In [ ]:
############### charts
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='white')
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go

############### common
import numpy as np


# Plotly Features

Plotly позволят создавать интерактивные графики (используя JS). Это, как правило, выглядит эффектно если вам необходимо сделать красивый график с которым можно взаимодействовать прямо в тетрадке.

In [ ]:
df = px.data.stocks()
fig = px.line(df, x="date", y=df.columns[1:], title='Stock prices')
fig.show()

# Slider


Если вам необходимо показать какие-то данные во времени, то можно использовать удобный функционал slider

https://plotly.com/python/sliders/

In [ ]:
df = px.data.gapminder()

fig = px.scatter(df, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
           size="pop", color="continent", hover_name="country",
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])

fig["layout"].pop("updatemenus")
fig.show()

# Custom sliding

Вы можете создать свою анимацию просто сделав множество последовательных картинок. Slider позволят вам это удобно упаковать.


In [ ]:
# https://plotly.com/python/sliders/
fig = go.Figure()

# Add traces, one for each slider step
for step in np.arange(0, 5, 0.1):
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#00CED1", width=6),
            name="𝜈 = " + str(step),
            x=np.arange(0, 10, 0.01),
            y=np.sin(step * np.arange(0, 10, 0.01))))

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

# Поверхности

In [ ]:
fig = go.Figure(data=[go.Surface(z=np.random.normal(0, 1, size=(25, 25)))])

fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

# Можем применить slider здесь

In [ ]:
SURFACE_SIZE = (100, 100)

base = np.zeros(SURFACE_SIZE)
noised_bases_collection = np.array([
    # в случайные позиции добавим -1 1 с очень маленькими вероятностями
    base + np.random.choice([-1, 0, 1], size=SURFACE_SIZE, p=[0.005, 0.99, 0.005]) for _ in range(10)
])

In [ ]:
fig = go.Figure()

for base_id in range(len(noised_bases_collection)):
    fig.add_trace(
        go.Surface(
            visible=False,
            name="𝜈 = " + str(base_id),
            z=noised_bases_collection[base_id]
        )
    )

fig.data[0].visible = True

steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

# Heatmap/Imshow `Seaborn` vs `Matplotlib`

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable


ax = plt.subplot(111)
im = ax.imshow(np.arange(100).reshape(10, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

_ = plt.colorbar(im, cax=cax)  # color bar добавлять заморочено сильно

In [ ]:
_ = sns.heatmap(np.arange(100).reshape(10, 10))

# Boxplots/Violinplot

In [ ]:
import pandas as pd

In [ ]:
iris = sns.load_dataset('iris')

In [ ]:
iris.head(2)

In [ ]:
_ = sns.boxplot(x='variable', y='value', palette=["m", "g"],
                data=pd.melt(iris.drop('species', axis=1)))

In [ ]:
_ = sns.violinplot(x='variable', y='value', palette=["m", "g"],
                   data=pd.melt(iris.drop('species', axis=1)))

In [ ]:
_ = sns.heatmap(iris.drop('species', axis=1).corr())

In [ ]:
_ = sns.pairplot(iris)

# `Scikit-learn` тоже имеет немного каноничных наборов данных

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
# create a dataFrame
data = load_iris()
X, y = data.data, data.target
df = pd.DataFrame(data=data['data'], columns=data['feature_names'])
df['y'] = data['target']

In [ ]:
df.head()

In [ ]:
# сделаем колонку со строковым описанием классов
df['class'] = df['y'].map({i: item for i, item in enumerate(data['target_names'])})

In [ ]:
fig = px.scatter_3d(df, x='sepal length (cm)', y='sepal width (cm)', z='petal width (cm)',
                    color='class')
fig.show()

# PCA

In [ ]:
from sklearn import decomposition

In [ ]:
pca = decomposition.PCA(n_components=2)

X_centered = X - X.mean(axis=0)
pca.fit(X_centered)
X_pca = pca.transform(X_centered)

In [ ]:
plt.plot(X_pca[y == 0, 0], X_pca[y == 0, 1], 'bo', label='Setosa')
plt.plot(X_pca[y == 1, 0], X_pca[y == 1, 1], 'go', label='Versicolour')
plt.plot(X_pca[y == 2, 0], X_pca[y == 2, 1], 'ro', label='Virginica')
plt.legend()

# Пример, когда PCA не лучший выбор

In [ ]:
from sklearn.datasets import load_digits

In [ ]:
digits = load_digits()
X = digits.data
y = digits.target

In [ ]:
plt.figure(figsize=(16, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X[i,:].reshape([8,8]));

In [ ]:
pca = decomposition.PCA(n_components=2)
X_reduced = pca.fit_transform(X)

print('Projecting %d-dimensional data to 2D' % X.shape[1])

plt.figure(figsize=(12,10))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y, 
            edgecolor='none', alpha=0.7, s=40,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))
plt.colorbar()
plt.title('MNIST. PCA projection')

# TSNE преобразование

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=17)

# tsne.fit(X)

X_tsne = tsne.fit_transform(X)

plt.figure(figsize=(12,10))
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, 
            edgecolor='none', alpha=0.7, s=40,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))
plt.colorbar()
plt.title('MNIST. t-SNE projection')